Core Idea:

Idea 1: The origins of cinema gross revenue increase. Do we like
movies this much more or just have more money to spend?
According to the data provided in the dataset, the gross revenue
of movies per year has been steadily increasing. First assumptions
can lead us to believe that movies just became bigger and more
popular - but the world's economic growth can actually play a big
role in this increase. The gross revenue data can be therefore
compared with various economical growth indicators: world's GDP
(world's bank data), indexes of world's biggest markets (ie. data
from DOW, NQ) etc. It would also be interesting to check whether
the correletion in growth - if present - diverges between
different world regions? Are there regions where the correletion
between the two is significantly stronger? As a bonus to this idea
on gross revenue: check whether it's increase is proportional to
the growing budget needed to produce and market the movies.

**Possible Expansions upon the idea**:

Genre and Revenue Analysis: Delve into how different movie genres fare in varying economic climates. Do certain genres thrive in economic downturns (e.g., escapism in fantasy or comedy) while others do better during booms (e.g., extravagant blockbusters)?

Audience Analysis: Look into the changing demographics of movie audiences over time. Has the increase in revenue been driven by a broader audience base, or by the same audience spending more?

Impact of Streaming and Digital Distribution: Consider the impact of digital distribution and streaming services on movie revenues. How has the shift from traditional theatrical releases to online platforms affected overall revenue?


Economic Factors and Consumer Spending Habits: Explore how broader economic factors like disposable income levels and unemployment rates correlate with movie spending. This can shed light on whether people spend more on movies when they have more disposable income or use movies as an affordable entertainment option during tougher times.



# Project Milestone 2 - Matea

This notebook contians initial data exploration and analysis for the Project Milestone 2
____________________________

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

### Loading the Data

In [2]:
# main data path and relative paths to the datasets
DATA_PATH = "data/"
DATA_PATH_MOVIESUMMARIES = DATA_PATH + "MovieSummaries/"
DATA_PATH_MOVIELENS = DATA_PATH + "MovieLens/"
DATA_PATH_IMDB = DATA_PATH + "IMDBData/"

DATASET_PATH = {
    "movie_metadata": DATA_PATH_MOVIESUMMARIES + "movie.metadata.tsv",
    "movie_budget": DATA_PATH_MOVIELENS + "movies_metadata.csv",
    "imdb_ratings": DATA_PATH_IMDB + "title.ratings.tsv",
    "imdb_basics": DATA_PATH_IMDB + "title.basics.tsv",
}
DATASET_COLUMNS = {
    "movie_metadata": [
        "wikipedia_id",
        "freebase_movie_id",
        "title",
        "release_date",
        "revenue",
        "runtime",
        "languages",
        "countries",
        "genres",
    ],
    "movie_budget": [
        "budget",
        "imdb_id",
        "original_title",
        "popularity",
        "release_date",
        "revenue",
        "runtime",
        "title",
        "vote_average",
        "vote_count",
    ],
    "imdb_ratings": ["imdb_id", "imdb_rating", "num_votes"],
    "imdb_basics": [
        "imdb_id",
        "title_type",
        "primary_title",
        "title",
        "is_adult",
        "year",
        "end_year",
        "runtime",
        "genres",
    ],
}

TODO: Add dtypes for all datasets

In [3]:
DATASET_TYPES = {
    "movie_metadata": {
        "wikipedia_id": "string",
        "freebase_movie_id": "string",
        "title": "string",
        "release_date": "string",
        "revenue": "float64",
        "runtime": "string",
        "languages": "object",
        "countries": "object",
        "genres": "object",
    },
    "movie_budget": {
        "budget": "object",
        "imdb_id": "string",
        "original_title": "object",
        "popularity": "object",
        "release_date": "object",
        "revenue": "float64",
        "runtime": "string",
        "title": "object",
        "vote_average": "float64",
        "vote_count": "float64",
    },
    "imdb_ratings": {
        "imdb_id": "string",
        "imdb_rating": "float64",
        "num_votes": "int64",
    },
    "imdb_basics": {
        "imdb_id": "string",
        "title_type": "string",
        "primary_title": "string",
        "title": "string",
        "is_adult": "string",
        "year": "string",
        "end_year": "string",
        "runtime": "string",
        "genres": "string",
    },
}

In [4]:
# load the datasets as pandas dataframes
movie_metadata = pd.read_csv(
    DATASET_PATH["movie_metadata"],
    sep="\t",
    names=DATASET_COLUMNS["movie_metadata"],
    index_col=False,
    header=None,
    dtype=DATASET_TYPES["movie_metadata"],
)
movie_budget = pd.read_csv(
    DATASET_PATH["movie_budget"],
    sep=",",
    usecols=DATASET_COLUMNS["movie_budget"],
    index_col=False,
    header=0,
    dtype=DATASET_TYPES["movie_budget"],
)
imdb_ratings = pd.read_csv(
    DATASET_PATH["imdb_ratings"],
    sep="\t",
    names=DATASET_COLUMNS["imdb_ratings"],
    index_col=False,
    header=0,
    dtype=DATASET_TYPES["imdb_ratings"],
)
imdb_basics = pd.read_csv(
    DATASET_PATH["imdb_basics"],
    sep="\t",
    names=DATASET_COLUMNS["imdb_basics"],
    index_col=False,
    header=0,
    dtype=DATASET_TYPES["imdb_basics"],
)

In [5]:
# Print the shapes of the datasets
print("Movie Metadata Shape: ", movie_metadata.shape)
print("MovieLens Budget Shape: ", movie_budget.shape)
print("IMDB Ratings Shape: ", imdb_ratings.shape)
print("IMDB Basics Shape: ", imdb_basics.shape)

Movie Metadata Shape:  (81741, 9)
MovieLens Budget Shape:  (45466, 10)
IMDB Ratings Shape:  (1368131, 3)
IMDB Basics Shape:  (10309011, 9)


#### Data Handling

***`movie_metadata`***

In [6]:
# print percentage of nan values in each column
print(movie_metadata.isna().sum() / len(movie_metadata) * 100)

wikipedia_id          0.000000
freebase_movie_id     0.000000
title                 0.000000
release_date          8.443743
revenue              89.722416
runtime              25.018045
languages             0.000000
countries             0.000000
genres                0.000000
dtype: float64


In [7]:
# since we're mainly interested in the box office revenue, we drop all rows where this value is missing
movie_metadata = movie_metadata.dropna(subset=["revenue"])
# also, we want to merge the cmu movies dataset with the imbds dataset on the moevie name, year and runtime
# that's why we need to drop the rows with missing values in these columns as well
movie_metadata = movie_metadata.dropna(subset=["release_date"])
movie_metadata = movie_metadata.dropna(subset=["runtime"])

# runtime was converted from float to string, so the ".0" needs to be removed
movie_metadata["runtime"] = movie_metadata["runtime"].astype(str).str.replace(".0", "")

# make a column year extracting the year from the release date
movie_metadata["year"] = movie_metadata["release_date"].str[:4]

In [8]:
# see if there are any duplicates in the movie metadata dataset
display(
    movie_metadata[
        movie_metadata.duplicated(
            subset=["title", "release_date", "runtime"], keep=False
        )
    ].sort_values(by=["title", "release_date", "runtime"])
)

,wikipedia_id,freebase_movie_id,title,release_date,revenue,runtime,languages,countries,genres,year


In [10]:
movie_metadata.head()

,wikipedia_id,freebase_movie_id,title,release_date,revenue,runtime,languages,countries,genres,year
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",2001
7,10408933,/m/02qc0j7,Alexander's Ragtime Band,1938-08-16,3600000.0,106,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/04t36"": ""Musical"", ""/m/01z4y"": ""Comedy"", ...",1938
13,171005,/m/016ywb,Henry V,1989-11-08,10161099.0,137,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/04xvh5"": ""Costume drama"", ""/m/082gq"": ""Wa...",1989
17,77856,/m/0kcn7,Mary Poppins,1964-08-27,102272727.0,139,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0hj3myq"": ""Children's/Family"", ""/m/04t36""...",1964
21,612710,/m/02wjqm,New Rose Hotel,1999-10-01,21521.0,92,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",1999


***`movie_budget`***

In [11]:
movie_budget = movie_budget.replace("\\N", np.nan)
print(movie_budget.isna().sum() / len(movie_budget) * 100)

budget            0.000000
imdb_id           0.037391
original_title    0.000000
popularity        0.010997
release_date      0.191352
revenue           0.013197
runtime           0.578454
title             0.013197
vote_average      0.013197
vote_count        0.013197
dtype: float64


In [15]:
movie_budget = movie_budget.dropna(subset=["revenue"])
movie_budget = movie_budget.dropna(subset=["release_date"])
movie_budget = movie_budget.dropna(subset=["runtime"])
movie_budget = movie_budget.dropna(subset=["imdb_id"])

# make a column year extracting the year from the release date
movie_budget["year"] = movie_budget["release_date"].str[:4]

In [18]:
display(
    movie_budget[movie_budget.duplicated(subset=["imdb_id"], keep=False)]
    .sort_values(by=["imdb_id"])
    .head(10)
)

,budget,imdb_id,original_title,popularity,release_date,revenue,runtime,title,vote_average,vote_count,year
38871,0,tt0022537,The Viking,0.002362,1931-06-21,0.0,70.0,The Viking,0.0,0.0,1931
16167,0,tt0022537,The Viking,0.002362,1931-06-21,0.0,70.0,The Viking,0.0,0.0,1931
949,4,tt0022879,A Farewell to Arms,1.914697,1932-12-08,25.0,89.0,A Farewell to Arms,6.2,29.0,1932
15074,4,tt0022879,A Farewell to Arms,2.411191,1932-12-08,25.0,89.0,A Farewell to Arms,6.2,29.0,1932
838,0,tt0046468,Tsuma,0.096079,1953-04-29,0.0,89.0,Wife,0.0,0.0,1953
30001,0,tt0046468,Tsuma,0.619388,1953-04-29,0.0,89.0,Wife,0.0,0.0,1953
7345,0,tt0062229,Le Samouraï,9.091288,1967-10-25,39481.0,105.0,Le Samouraï,7.9,187.0,1967
9165,0,tt0062229,Le Samouraï,9.091288,1967-10-25,39481.0,105.0,Le Samouraï,7.9,187.0,1967
29374,0,tt0067306,King Lear,0.187901,1971-02-04,0.0,137.0,King Lear,8.0,3.0,1971
15702,0,tt0067306,King Lear,0.187901,1971-02-04,0.0,137.0,King Lear,8.0,3.0,1971


In [19]:
# drop duplicates since there are some movies with the same imdb id
movie_budget = movie_budget.drop_duplicates(subset=["imdb_id"])

In [20]:
movie_budget.head()

,budget,imdb_id,original_title,popularity,release_date,revenue,runtime,title,vote_average,vote_count,year
0,30000000,tt0114709,Toy Story,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,1995
1,65000000,tt0113497,Jumanji,17.015539,1995-12-15,262797249.0,104.0,Jumanji,6.9,2413.0,1995
2,0,tt0113228,Grumpier Old Men,11.7129,1995-12-22,0.0,101.0,Grumpier Old Men,6.5,92.0,1995
3,16000000,tt0114885,Waiting to Exhale,3.859495,1995-12-22,81452156.0,127.0,Waiting to Exhale,6.1,34.0,1995
4,0,tt0113041,Father of the Bride Part II,8.387519,1995-02-10,76578911.0,106.0,Father of the Bride Part II,5.7,173.0,1995


In [22]:
movie_budget["budget"].value_counts()  ##FIX THIS!! 0 = missing

budget
0           36223
5000000       286
10000000      258
20000000      243
2000000       241
            ...  
12899867        1
1601792         1
558000          1
474700          1
1549000         1
Name: count, Length: 1219, dtype: int64

***`imdb_ratings`***

In [23]:
imdb_ratings = imdb_ratings.replace("\\N", np.nan)
print(imdb_ratings.isna().sum() / len(imdb_ratings) * 100)

imdb_id        0.0
imdb_rating    0.0
num_votes      0.0
dtype: float64


In [25]:
display(
    imdb_ratings[imdb_ratings.duplicated(subset=["imdb_id"], keep=False)]
    .sort_values(by=["imdb_id"])
    .head(10)
)

,imdb_id,imdb_rating,num_votes


***`imdb_basics`***

In [24]:
imdb_basics = imdb_basics.replace("\\N", np.nan)
print(imdb_basics.isna().sum() / len(imdb_basics) * 100)

imdb_id           0.000000
title_type        0.000000
primary_title     0.000165
title             0.000165
is_adult          0.000010
year             13.401906
end_year         98.888060
runtime          69.875617
genres            4.464415
dtype: float64


In [26]:
display(
    imdb_basics[imdb_basics.duplicated(subset=["imdb_id"], keep=False)]
    .sort_values(by=["imdb_id"])
    .head(10)
)

,imdb_id,title_type,primary_title,title,is_adult,year,end_year,runtime,genres


In [27]:
# drop column end_year
imdb_basics = imdb_basics.drop(columns=["end_year"])

# drop rows with missing values, since we are joining the imdb_dataset on this keys later
imdb_basics = imdb_basics.dropna(subset=["year"])
imdb_basics = imdb_basics.dropna(subset=["title"])
imdb_basics = imdb_basics.dropna(subset=["runtime"])

### Merging

First we will merge the `imdb_ratings` and `imdb_basics` dataframes, in order to complete the imdb dataset.
Next, we will expand the newly created imdb dataset with the data from the movie budget dataset.
We will use the imdb_id column as a key, and we will use a left join, because we want to keep all the movies, even if they don't have a rating or a budget.

In [28]:
imdb_dataset_temp = pd.merge(imdb_basics, imdb_ratings, on="imdb_id", how="left")
imdb_dataset = pd.merge(
    imdb_dataset_temp,
    movie_budget,
    on="imdb_id",
    how="left",
    suffixes=("", "_movie_budget"),
)
display(imdb_dataset)

,imdb_id,title_type,primary_title,title,is_adult,year,runtime,genres,imdb_rating,num_votes,budget,original_title,popularity,release_date,revenue,runtime_movie_budget,title_movie_budget,vote_average,vote_count,year_movie_budget
0,tt0000001,short,Carmencita,Carmencita,0,1894,1,"Documentary,Short",5.7,2007.0,0,Carmencita,1.273072,1894-03-14,0.0,1.0,Carmencita,4.9,18.0,1894
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,5,"Animation,Short",5.8,269.0,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,4,"Animation,Comedy,Romance",6.5,1909.0,0,Pauvre Pierrot,0.673164,1892-10-28,0.0,4.0,Poor Pierrot,6.1,19.0,1892
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,12,"Animation,Short",5.5,178.0,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,1,"Comedy,Short",6.2,2691.0,0,Blacksmith Scene,1.061591,1893-05-08,0.0,1.0,Blacksmith Scene,5.8,19.0,1893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2909426,tt9916754,movie,Chico Albuquerque - Revelações,Chico Albuquerque - Revelações,0,2013,49,Documentary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN
2909427,tt9916766,tvEpisode,Episode #10.15,Episode #10.15,0,2019,43,"Family,Game-Show,Reality-TV",7.0,22.0,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN
2909428,tt9916840,tvEpisode,Horrid Henry's Comic Caper,Horrid Henry's Comic Caper,0,2014,11,"Adventure,Animation,Comedy",8.8,6.0,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN
2909429,tt9916856,short,The Wind,The Wind,0,2015,27,Short,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN


In [29]:
imdb_dataset = imdb_dataset.loc[
    :, ~imdb_dataset.columns.str.endswith("_movie_budget")
]  # keep only the columns from the imdb dataset

Next, we will merge the imdb_dataset containing the budget, with the movies dataset. We will mege on the `movie_name`, `year` and `movie_runtime` columns. 

But before we do that, we need to check and fix the type of the columns that we will be merging.

In [30]:
print("type of column year in imdb", type(imdb_dataset["year"][0]))
print("type of column year in movies", type(movie_metadata["year"][0]))

print("type of column movie_name in imdb", type(imdb_dataset["title"][0]))
print("type of column movie_name in movies", type(movie_metadata["title"][0]))

print("type of column movie_runtime in imdb", type(imdb_dataset["runtime"][0]))
print("type of column movie_runtime in movies", type(movie_metadata["runtime"][0]))

type of column year in imdb <class 'str'>
type of column year in movies <class 'str'>
type of column movie_name in imdb <class 'str'>
type of column movie_name in movies <class 'str'>
type of column movie_runtime in imdb <class 'str'>
type of column movie_runtime in movies <class 'str'>


In [31]:
movies_merged = pd.merge(
    movie_metadata,
    imdb_dataset,
    on=["title", "year", "runtime"],
    how="left",
)

In [32]:
display(movies_merged)

,wikipedia_id,freebase_movie_id,title,release_date_x,revenue_x,runtime,languages,countries,genres_x,year,...,genres_y,imdb_rating,num_votes,budget,original_title,popularity,release_date_y,revenue_y,vote_average,vote_count
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",2001,...,"Action,Horror,Sci-Fi",4.9,56918.0,28000000,Ghosts of Mars,7.058599,2001-08-24,14010832.0,4.8,299.0
1,10408933,/m/02qc0j7,Alexander's Ragtime Band,1938-08-16,3600000.0,106,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/04t36"": ""Musical"", ""/m/01z4y"": ""Comedy"", ...",1938,...,"Drama,Music,Musical",6.8,2268.0,2000000,Alexander's Ragtime Band,0.632261,1938-05-24,4000000.0,4.8,6.0
2,171005,/m/016ywb,Henry V,1989-11-08,10161099.0,137,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/04xvh5"": ""Costume drama"", ""/m/082gq"": ""Wa...",1989,...,"Biography,Drama,History",7.5,31200.0,9000000,Henry V,7.307967,1989-10-05,0.0,7.4,73.0
3,77856,/m/0kcn7,Mary Poppins,1964-08-27,102272727.0,139,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0hj3myq"": ""Children's/Family"", ""/m/04t36""...",1964,...,"Comedy,Family,Fantasy",7.8,181780.0,6000000,Mary Poppins,15.11549,1964-08-27,102272727.0,7.4,1217.0
4,612710,/m/02wjqm,New Rose Hotel,1999-10-01,21521.0,92,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",1999,...,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8241,1191380,/m/04f_y7,Wilde,1997,2158775.0,118,"{""/m/02h40lc"": ""English Language""}","{""/m/014tss"": ""Kingdom of Great Britain"", ""/m/...","{""/m/0hn10"": ""LGBT"", ""/m/017fp"": ""Biography"", ...",1997,...,"Biography,Drama,Romance",6.9,17900.0,0,Wilde,3.895735,1997-09-01,0.0,6.7,62.0
8242,54540,/m/0f7hw,Coming to America,1988-06-29,288752301.0,117,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06cvj"": ""Romantic comedy"", ""/m/03p5xs"": ""...",1988,...,"Comedy,Romance",7.1,218904.0,39000000,Coming to America,7.830087,1988-06-28,288752301.0,6.5,944.0
8243,7761830,/m/0kvgqb,Spaced Invaders,1990,15369573.0,100,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0hj3mt0"": ""Alien Film"", ""/m/06n90"": ""Scie...",1990,...,"Adventure,Comedy,Sci-Fi",5.3,3885.0,3000000,Spaced Invaders,2.428279,1990-04-27,15369573.0,4.8,27.0
8244,1918494,/m/0660qx,State and Main,2000-08-26,6944471.0,106,"{""/m/02bjrlw"": ""Italian Language"", ""/m/02h40lc...","{""/m/0f8l9c"": ""France"", ""/m/09c7w0"": ""United S...","{""/m/0gf28"": ""Parody"", ""/m/09n5t_"": ""Americana...",2000,...,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
movies_merged.isna().sum() / len(movies_merged) * 100

wikipedia_id          0.000000
freebase_movie_id     0.000000
title                 0.000000
release_date_x        0.000000
revenue_x             0.000000
runtime               0.000000
languages             0.000000
countries             0.000000
genres_x              0.000000
year                  0.000000
imdb_id              54.244482
title_type           54.244482
primary_title        54.244482
is_adult             54.244482
genres_y             54.244482
imdb_rating          54.268736
num_votes            54.268736
budget               58.452583
original_title       58.452583
popularity           58.452583
release_date_y       58.452583
revenue_y            58.452583
vote_average         58.452583
vote_count           58.452583
dtype: float64

### Load Financial Data

...